In [1]:
!pip -q install huggingface chromadb transformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
!pip install configparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import configparser
import os
api_reader = configparser.ConfigParser()
api_reader.read_file(open('/content/apidata.config'))

os.environ['HUGGINGFACEHUB_API_TOKEN'] = api_reader["OPENAI"]["HFREAD"]

In [4]:
with open('/content/linux_play.txt') as lin:
  txt_lin = lin.read()

In [5]:
from langchain import text_splitter as ts

In [ ]:
#Recursive splitter is recommended splitter

#By default the characters it tries to split on are ["\n\n", "\n", " ", ""]

#length_function: how the length of chunks is calculated. Defaults to just counting number of characters, but it’s pretty common to pass a token counter here.

#chunk_size: the maximum size of your chunks (as measured by the length function).

#chunk_overlap: the maximum overlap between chunks. It can be nice to have some overlap to maintain some continuity between chunks (eg do a sliding window).

In [6]:
rec_splitter = ts.RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

In [7]:
split_lin = rec_splitter.create_documents([txt_lin])
print(len(split_lin))

152


In [8]:
print(split_lin[0])

page_content='BEGIN;\nLinux Playbook' lookup_str='' metadata={} lookup_index=0


In [9]:
rec_500splitter = ts.RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=len
)

In [10]:
split_500lin = rec_500splitter.create_documents([txt_lin])
print(len(split_500lin))

29


In [11]:
split_500lin[0]

Document(page_content='BEGIN;\nLinux Playbook\n\nThe command and scenarios has to be executed inside the\nKali Docker image. The docker image is called linux_playg.\nThe docker will be contain the\nset of set of files, prepared for this plabook and\nuploaded on to dockerhub.', lookup_str='', metadata={}, lookup_index=0)

In [12]:
#Lets work on the character splitter

#How the text is split: by single character

#How the chunk size is measured: by length function passed in (defaults to number of characters)

In [15]:
char_split = ts.CharacterTextSplitter(
    separator="\n\n",
    chunk_size=500,
    chunk_overlap=200,
    length_function=len
)

In [16]:
char_500lin = char_split.create_documents([txt_lin])
print(len(char_500lin))

19


In [17]:
char_500lin[0]

Document(page_content='BEGIN;\nLinux Playbook\n\nThe command and scenarios has to be executed inside the\nKali Docker image. The docker image is called linux_playg.\nThe docker will be contain the\nset of set of files, prepared for this plabook and\nuploaded on to dockerhub.', lookup_str='', metadata={}, lookup_index=0)

In [ ]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [20]:
hf_splitter = ts.CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, 
                                                                  chunk_size=100, 
                                                                  chunk_overlap=0)
texts = hf_splitter.split_text(txt_lin)

In [ ]:
#Interesting latex splitter 

In [22]:
latex_text = """
\documentclass{article}

\begin{document}

\maketitle

\section{Introduction}
Large language models (LLMs) are a type of machine learning model that can be trained on vast amounts of text data to generate human-like language. In recent years, LLMs have made significant advances in a variety of natural language processing tasks, including language translation, text generation, and sentiment analysis.

\subsection{History of LLMs}
The earliest LLMs were developed in the 1980s and 1990s, but they were limited by the amount of data that could be processed and the computational power available at the time. In the past decade, however, advances in hardware and software have made it possible to train LLMs on massive datasets, leading to significant improvements in performance.

\subsection{Applications of LLMs}
LLMs have many applications in industry, including chatbots, content creation, and virtual assistants. They can also be used in academia for research in linguistics, psychology, and computational linguistics.

\end{document}
"""
latex_splitter = ts.LatexTextSplitter(chunk_size=400, 
                                      chunk_overlap=0)

In [23]:
docs = latex_splitter.create_documents([latex_text])
docs

[Document(page_content='\\documentclass{article}\n\n\x08egin{document}\n\n\\maketitle', lookup_str='', metadata={}, lookup_index=0),
 Document(page_content='Introduction}\nLarge language models (LLMs) are a type of machine learning model that can be trained on vast amounts of text data to generate human-like language. In recent years, LLMs have made significant advances in a variety of natural language processing tasks, including language translation, text generation, and sentiment analysis.', lookup_str='', metadata={}, lookup_index=0),
 Document(page_content='History of LLMs}\nThe earliest LLMs were developed in the 1980s and 1990s, but they were limited by the amount of data that could be processed and the computational power available at the time. In the past decade, however, advances in hardware and software have made it possible to train LLMs on massive datasets, leading to significant improvements in performance.', lookup_str='', metadata={}, lookup_index=0),
 Document(page_cont

In [24]:
from langchain.text_splitter import MarkdownTextSplitter

markdown_text = """
# 🦜️🔗 LangChain

⚡ Building applications with LLMs through composability ⚡

## Quick Install

```bash
# Hopefully this code block isn't split
pip install langchain
```

As an open source project in a rapidly developing field, we are extremely open to contributions.
"""
markdown_splitter = MarkdownTextSplitter(chunk_size=100, 
                                         chunk_overlap=0)
mar_docs = markdown_splitter.create_documents([markdown_text])
mar_docs


[Document(page_content='# 🦜️🔗 LangChain\n\n⚡ Building applications with LLMs through composability ⚡', lookup_str='', metadata={}, lookup_index=0),
 Document(page_content="Quick Install\n\n```bash\n# Hopefully this code block isn't split\npip install langchain", lookup_str='', metadata={}, lookup_index=0),
 Document(page_content='As an open source project in a rapidly developing field, we are extremely open to contributions.', lookup_str='', metadata={}, lookup_index=0)]

In [26]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [27]:
nltk_splitter = ts.NLTKTextSplitter(chunk_size=1000)

txt_nltk = nltk_splitter.create_documents([txt_lin])

In [ ]:
#there is Python code splitter too

In [30]:
!pip -q install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.7 MB/s eta 0:00:00


In [31]:
tik_spliter = ts.TokenTextSplitter(chunk_size=50,
                                   chunk_overlap = 0)

tik_lin = tik_spliter.split_text(txt_lin)
print(len(tik_lin))

72
